In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

**Machine Learning Clássico - Data Cleaning**

In [94]:
## Importando datasets
test = pd.read_csv('/kaggle/input/titanic/test.csv')
train = pd.read_csv('/kaggle/input/titanic/train.csv')

## Criando copia do banco de dados de treino
train_c = train.copy()

train.describe(include = 'all')

In [95]:
## Verificando dados
print('Train data types')
print(train.info())
print('-'*50)
print('Test data types')
print(test.info())

In [96]:
## Contando os valores NaN nas features

print('Train data NaN Values')
print(train.isnull().sum())
print('-'*20)
print('Test data NaN Values')
print(test.isnull().sum())

**Data Cleaning**

In [97]:
## Retirando as features sem utilidade
for data in [train_c, test]:
    data = data.drop(['Cabin','Ticket'], axis = 1, inplace = True)

## Retirando"PassengerId" do banco de dados de teste
train_c = train_c.drop(["PassengerId"], axis = 1)

In [98]:
#Preenchendo valores nulos
for data in [train_c, test]:
    data['Age'].fillna(data['Age'].median(), inplace = True)
    data['Fare'].fillna(data['Fare'].median(), inplace = True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)
    
print('Train data NaN Values')
print(train_c.isnull().sum())
print('-'*20)
print('Test data NaN Values')
print(test.isnull().sum())

**Aperfeiçoamento da modelagem**

In [99]:
for data in [train_c,test]:
    ## Separando feature "Age" em 5 intervalos de mesmo tamanho
    data['Age_group'] = pd.cut(data['Age'].astype(int), 5)
    
    ## Separando feature "Fare" em 6 intervalos com mesmo numero de dados
    data['Fare_group'] = pd.qcut(data['Fare'], 6)

In [100]:
## Relembrando quais Features são categóricas
train_c.info()

**Encoding de Variáveis Categóricas**

In [101]:
## One-Hot-encode
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse = False)

## Função para realizar o encoding das variáveis
def OH_encoder(data,cols):
    data_encoded = data.copy()
    for col in cols:      
        ## Criando colunas para atribuir One-hot-encode
        OH_cols = pd.DataFrame(onehot.fit_transform(data[[col]]),dtype = 'int')
        
        ## Nomeando as colunas
        OH_cols.columns = onehot.get_feature_names([col])
        
        ## Adicionando as novas colunas codificadas ao dataframe
        data_encoded = data_encoded.drop([col], axis = 1)
        data_encoded = pd.concat([data_encoded,OH_cols], axis = 1)
    return data_encoded

#Realizando o encoding dos dataframes
train_encoded = OH_encoder(train_c,["Sex","Embarked"])
test_encoded = OH_encoder(test,["Sex","Embarked"])

In [102]:
## Label encoding
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

for data in [train_encoded,test_encoded]:
    data["Age_group_code"] = label.fit_transform(data["Age_group"])
    data["Fare_group_code"] = label.fit_transform(data["Fare_group"])
    data = data.drop(["Age_group","Fare_group","Name"], axis = 1, inplace = True)
    
train_encoded.head()

**Avaliando precisão do modelo**

In [103]:
## Criando e avaliando modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

randomforest_model1 = RandomForestClassifier(random_state = 0)

X_model1 = train_encoded.drop('Survived', axis = 1)
y_model1 = train_encoded['Survived']

X_train,X_valid,y_train,y_valid = train_test_split(X_model1,y_model1,test_size = 0.2,random_state = 0)

randomforest_model1.fit(X_train,y_train)
preds = randomforest_model1.predict(X_valid)

accuracy = round(accuracy_score(preds,y_valid)*100,2)
print(accuracy)

**Criando Pipelines**

In [104]:
## Restaurando valores nulos originais do dataframe train_c nas colunas "Age" e "Embarked"
train_c_na = train_c.copy()
train_c_na = train_c_na.drop(["Age","Embarked"], axis = 1)
train_c_na = pd.concat([train_c_na,train[["Age","Embarked"]]],axis = 1)

train_c_na.isnull().sum()

**Função para avaliar estratégias**

In [105]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def pipeline_maker(data, encoder, model, numerical_imputer = SimpleImputer(),
                   categorical_imputer = SimpleImputer(strategy = 'most_frequent')):
    
    X_pipe = data.drop('Survived', axis = 1)
    y_pipe = data['Survived']

    X_train_pipe,X_valid_pipe,y_train_pipe,y_valid_pipe = train_test_split(X_pipe,y_pipe,
                                                                           test_size = 0.2,
                                                                           random_state = 0)

    #Criando os passos do preprocessor, para features numéricas e categóricas
    num_transf = numerical_imputer
    cat_transf = Pipeline(steps = [('imputer', categorical_imputer),
                                               ('encoder',encoder)])
    preprocessor = ColumnTransformer(transformers = [
                                                ('num',num_transf,['Age']),
                                                ('cat',cat_transf,['Age_group','Fare_group',
                                                                          'Sex','Embarked'])])
    
    #Criando pipeline e gerando predições
    pipe = Pipeline(steps = [('preprocessor', preprocessor),
                                         ('model', model)])
    pipe.fit(X_train_pipe,y_train_pipe)
    preds_pipe = pipe.predict(X_valid_pipe)

    #Avaliando as predições
    accuracy = round(accuracy_score(preds_pipe,y_valid_pipe)*100,2)
    return accuracy, preds_pipe

**Definindo estratégias**

In [106]:
## Imputers
mean_imputer = SimpleImputer()
mostfreq_imputer = SimpleImputer(strategy = 'most_frequent')
median_imputer = SimpleImputer(strategy = 'median')
zero_imputer = SimpleImputer(strategy = 'constant', fill_value = 0)
imputers = [mean_imputer, mostfreq_imputer, median_imputer, zero_imputer]

## Encoders
from sklearn.preprocessing import OrdinalEncoder
onehot_encoder = OneHotEncoder(sparse = False)
ordinal_encoder = OrdinalEncoder()
encoders = [onehot_encoder, ordinal_encoder]

#model
randomforest = RandomForestClassifier(random_state = 0)

**Teste de estratégias**

In [107]:
for encoder in encoders:
    for imputer in imputers:
        accuracy,_ = pipeline_maker(train_c_na, encoder, randomforest, numerical_imputer = imputer)
        print("pipe_"+str(encoder)+'_'+str(imputer)+'='+str(accuracy))

**Avalição dos testes**

Note que nenhuma dessas estratégias usadas separadamente, gerou uma precisão do modelo maior que suas utilizações mescladas.

**Cross Validation e Gradient Boosting**

In [108]:
## Cross Validation
from sklearn.model_selection import cross_val_score
## Criando função para gerar avaliação
def cross_validation(model, X, y):
    scores = cross_val_score(model, X, y, cv = 5, scoring = 'accuracy')
    return round(scores.mean()*100,2)

## Gerando avaliação do modelo mesclado 
accuracy_crossval = cross_validation(randomforest_model1, X_model1, y_model1)
print(accuracy_crossval)

In [109]:
## Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

## Criando Função Gradient Boosting classifier para gerar e avaliar um modelo 
def make_gbc(X,y):
    gbclass = GradientBoostingClassifier(random_state = 0, n_iter_no_change = 100) 
    gbclass.fit(X,y)
    score = cross_validation(gbclass, X, y)
    return score, gbclass

X_model2 = X_model1
y_model2 = y_model1

score,gbclass = make_gbc(X_model2,y_model2)
print(score)

**Feature Engineering**

**Realizando MI Score:**

In [110]:
## Obtendo a pontuação MI de cada feature
from sklearn.feature_selection import mutual_info_classif

## Definindo as funções discretas
discrete_features = train_encoded.drop(['Age','Fare'], axis = 1)

## Fução para fazer gráficos de MI score
def make_MI (X, y, discrete_features):
    
    ## Produzindo MI scores
    mi_scores = mutual_info_classif(X, y, discrete_features = discrete_features )
    mi_scores = pd.Series(mi_scores, index = X.columns)
    mi_scores = mi_scores.sort_values(ascending = False)
    
    ## Prduzindo gráfico
    sns.set_style('whitegrid')
    scores = mi_scores.sort_values(ascending = True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    
    return mi_scores

X = train_encoded.drop(['Survived'],axis = 1)
y = train_encoded['Survived']

make_MI(X, y, discrete_features)

**Analisando Resultados do MI score**

**Distribuição da feature "Fare" e relação dela com "Survived"**

In [111]:
fig, axs = plt.subplots(1,2, figsize = (18,7))

## Criando gráficos da distribuição de "Fare"
axs[0].set_title('Distribuição das Taxas')
sns.histplot(x = 'Fare', kde = True, data = train_c, ax = axs[0]);
axs[1].set_title('Relação Entre Taxas e Sobreviventes ')
sns.histplot(x = 'Fare',hue = 'Survived', kde = True, data = train_c, ax = axs[1])

Pelo gráfico "Distribuição das Taxas" é possível perceber que a maioria das taxas foram de menos de 50 dolares, contudo, ao passar deste valor, a partir do gráfico "Relação Entre Taxas e Sobreviventes", pode-se notar que o número de sobrevivêntes supera, em média, o número de mortes. Outro fato que vale a oenas ser destacado, é que "Fare" não possui distribuição normal.

**Avaliar a influência do sexo e idade do passageiro, na chance de sobrevivencia**

In [112]:
fig, axs = plt.subplots(2,2, figsize = (18,7), sharex = True, sharey = True)

## Criando gráficos da distribuição de "Fare"
axs[0,0].set_title('Sexo em relação à idade')
sns.histplot(x = 'Age',hue = 'Sex', kde = True, data = train_c, ax = axs[0,0], palette = "YlOrRd")

axs[0,1].set_title('Distribuição de sobreviventes por idade, comparando os sexos')
sns.histplot(data = train_c.loc[train_c['Survived']==1], x = 'Age', hue = 'Sex', kde = True, ax = axs[0,1], palette = "YlOrRd_r")

axs[1,0].set_title('Distribuição de sobrevivência de homens')
sns.histplot(data = train_c.loc[train_c['Sex']=='male'], x = 'Age', hue = 'Survived', kde = True, ax = axs[1,0])

axs[1,1].set_title('Distribuição de sobrevivência de mulheres')
sns.histplot(data = train_c.loc[train_c['Sex']=='female'], x = 'Age', hue = 'Survived', kde = True, ax = axs[1,1])

**Criando novas features a partir de existentes**

In [113]:
## Adicionando "Name" à train_encoded
train_encoded = pd.concat([train_encoded,train_c['Name']], axis = 1)

for data in [train_encoded,test]:
    #U# tilizando o método string split do pandas para retirar informações de texto 
    data['Title'] = data['Name'].str.split(',',expand = True)[1].str.split('.',expand = True)[0]
    data.drop("Name", axis = 1, inplace = True)

## Demonstrando quais os valores da coluna
print(train_encoded['Title'].value_counts())

**Criando novas Features a partir de operações matemáticas**

In [114]:
for data in [train_encoded,test]:
    #Criando Feature para contar número de integrantes da família
    data['Familysize'] = data['SibSp'] + data['Parch']
    
    #Criando Feature para identificar se a pessoa está ou não sozinha no navio
    data['IsAlone'] = 1
    for i in range(len(data.index)):
        if data.loc[i,'Familysize'] != 0:
            data.loc[i,'IsAlone'] = 0

train_encoded.head()

**Clustering**

**Criar nova features a partir da interação entre "Fare" e "Age"**

In [115]:
## Visualizando a interação entre "Fare" e "Age"
sns.scatterplot(x='Fare',y='Age',data = train_c)

Pode-se perceber que existem tres passageiros que pagaram uma Taxa muito maior que o resto, mais de 500, neste caso, esses dados são considerados outliers, já que estão extremamente fora do padrão, logo devem ser inicialmente retirados para a criação dos clusters, depois, devem ser estudados para que sejam incluidos em um grupo, manualmente.

In [116]:
from sklearn.cluster import KMeans

#Criando dataframe com "Age" e "Fare" sem outliers
fare_age_group = train_c.loc[train_c['Fare'] < 500,['Fare','Age']]

#Criando feature a partir da interação entre "Age" e "Fare"
kmeans = KMeans(n_clusters = 6, random_state = 0)
fare_age_group['Fare_age_group'] = kmeans.fit_predict(fare_age_group)
fare_age_group['Fare_age_group'] = fare_age_group['Fare_age_group'].astype('int')

fare_age_group

In [117]:

#Adicionando feature criada ao banco de dados de treino
train_c['Fare_age_group'] = fare_age_group['Fare_age_group']

#Criando visualização para entendimento da nova feature
fig, axs = plt.subplots(1,2, figsize = (15,7))
sns.barplot(x = 'Fare_age_group', y = 'Survived', data = train_c, ax = axs[1], palette = "rainbow_r")
sns.scatterplot(x='Fare',y='Age', hue = 'Fare_age_group', data = train_c, ax = axs[0],palette = "rainbow_r" );

**Lidando com outliers**

In [118]:
#Descobrindo quem são so Outliers
train_c.loc[train_c['Fare_age_group'].isnull() == True,:]

In [119]:
#Selecionando um grupo para os Outliers
train_c.loc[train_c['Fare_age_group'].isnull() == True,['Fare_age_group']] = 1
train_c.loc[[258,679,737]]

In [120]:
#Adicionando a nova feature aos banco de dado train_encoded
train_encoded['Fare_age_group'] = train_c['Fare_age_group']
train_encoded

In [121]:
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

#Criando scaller 
scaller = MinMaxScaler(copy = False)

#Criando dataframe para gravar as features após realizar scalling
stand_data = pd.DataFrame()

#Loop para a realização do scalling
for column in ['Fare','Age']:
    stand_data[[column]] = train_c[[column]]
    stand_data[[column]] = scaller.fit_transform(stand_data[[column]])
    
#Gerando gráficos para mostrar a distribuição das features após o scalling 
fig, axs = plt.subplots(1,2, figsize = (18,7))

sns.histplot(x = 'Age', kde = True, data = stand_data, ax = axs[0]);
sns.histplot(x = 'Fare', kde = True, data = stand_data, ax = axs[1]);

In [122]:
from sklearn.decomposition import PCA

#Realizando o PCA de "Fare" e "Age" 
pca = PCA()
pca_data = pca.fit_transform(stand_data)

#Criando um dataframe para armazenar as features criadas a partir do PCA
columns = ['PC1','PC2']
pca_data = pd.DataFrame(pca_data, columns = columns)

#Criando dataframe para identificar como os eixos foram afetados pelo PCA
loadings = pd.DataFrame(pca.components_.T, columns = columns, index = ["Fare","Age"])
loadings

In [123]:
#Gerando MI scores das features PC1 e PC2
make_MI(pca_data, train_c["Survived"], discrete_features = False)

**Target Encoding**

In [125]:
from category_encoders import MEstimateEncoder

encoder = MEstimateEncoder(cols = ['Title'], m = 5)

title = encoder.fit_transform(train_encoded['Title'], train_encoded['Survived'])
title

In [126]:
fig, ax = plt.subplots(figsize = (9,7))

sns.distplot(train_encoded['Survived'], kde=False, norm_hist=True)
sns.kdeplot(title['Title'], color='r', ax=ax)
ax.set_xlabel("Survived")
ax.legend(labels=['Title', 'Survived']);

In [127]:
train_encoded['Title-encoded'] = title
train_encoded.info()

In [128]:
X_enc = train_encoded.loc[:,['Fare_age_group','Title-encoded']]
y_enc = train_encoded['Survived']
make_MI(X_enc, y_enc, discrete_features = False )